In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np  

/home/lmmista-wap266/anaconda3/envs/automl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the data 

In [2]:
data = pd.read_csv('train_new.csv')
train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)
train_data = TabularDataset(train_data)
valid_data = TabularDataset(valid_data)
# subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values

train_data.head(10)

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,RcNursEstDate,...,Harv_method,Harv_date,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield
2587,ID_1J0W3DPCM4H8,Vaishali,Mahua,20,12,TractorPlough FourWheelTracRotavator,2022-06-25,4,Manual_PuddledRandom,2022-06-02,...,hand,2022-11-04,800.0,2022-11-19,machine,30,10,plowed_in_soil,0.181818,280
2740,ID_2AYN21MOHJ0T,Nalanda,Rajgir,32,30,TractorPlough FourWheelTracRotavator,2022-07-10,6,Manual_PuddledRandom,2022-06-18,...,hand,2022-11-25,400.0,2023-02-23,machine,25,10,plowed_in_soil,0.187500,450
1961,ID_162UEZJWF00U,Nalanda,Rajgir,55,50,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-15,6,Manual_PuddledRandom,2022-06-26,...,hand,2022-12-15,1500.0,2023-01-20,machine,24,10,plowed_in_soil,0.718750,1800
2516,ID_8GSCZYF1YH2G,Nalanda,Rajgir,70,70,TractorPlough,2022-06-26,4,LineSowingAfterTillage,2022-06-30,...,machine,2022-12-02,NaN,2022-12-05,machine,28,40,plowed_in_soil,0.312500,560
2382,ID_1635AF76YBGK,Nalanda,Rajgir,200,195,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-12,3,Manual_PuddledRandom,2022-06-15,...,machine,2022-11-15,NaN,2022-11-15,machine,30,40,plowed_in_soil,0.625000,1250
765,ID_2FVBQXXYV589,Vaishali,Garoul,16,16,FourWheelTracRotavator,2022-07-13,6,LineSowingAfterTillage,2022-06-05,...,hand,2022-10-12,1500.0,2022-10-20,hand,19,10,plowed_in_soil,0.727273,1120
2806,ID_12X5MHEUNXX4,Vaishali,Mahua,1,1,TractorPlough,2022-06-22,4,Manual_PuddledRandom,2022-07-01,...,hand,2022-11-01,200.0,2022-11-11,hand,30,10,plowed_in_soil,0.045455,70
1316,ID_AOJWELMK62ZR,Vaishali,Mahua,12,8,TractorPlough FourWheelTracRotavator,2022-06-09,5,Manual_PuddledRandom,2022-06-14,...,hand,2022-10-15,600.0,2022-10-27,hand,30,10,plowed_in_soil,0.136364,6
2307,ID_Z5DM6T0SOBZC,Jamui,Jamui,14,14,TractorPlough,2022-08-06,4,Manual_PuddledRandom,2022-07-16,...,hand,2022-11-17,500.0,2023-01-18,machine,28,10,plowed_in_soil,0.227273,400
219,ID_NF99Y3VUGV1E,Jamui,Jamui,30,30,TractorPlough BullockPlough,2022-07-20,4,Manual_PuddledRandom,2022-06-18,...,hand,2022-11-28,90.0,2023-02-13,hand,29,10,plowed_in_soil,0.454545,800


In [3]:
# Test data without label.
print(type(train_data))

<class 'autogluon.core.dataset.TabularDataset'>


In [4]:
label = 'Yield'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    2476.000000
mean      568.142165
std       442.049203
min         4.000000
25%       300.000000
50%       430.000000
75%       729.750000
max      4050.000000
Name: Yield, dtype: float64


The mean root sqaured error is not part of the implemented metrics inside the AutoGluon    
Thus we need to implement this method by ourselves


In [5]:
from sklearn.metrics import make_scorer

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.sqaure(y_true - y_pred))) 

rmse_scorer = make_scorer(rmse, greater_is_better=False)

If you want to try with different metrics, remeber to change the metric parameter into different value

In [6]:
metric = 'root_mean_squared_error'  # specify the evaluation metric here, you can also use 'roc_auc', 'f1', etc.
predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets='best_quality',
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1, ag_args_fit={'num_gpus': 1}, # Most of the models used by AutoGluon support GPU training, including LightGBM, CatBoost, XGBoost and FastAI Neural Network.
    hyperparameters = {'NN_TORCH': {'num_epochs': 20}, 'GBM': {'num_boost_round': 40}} # last  argument is just for quick demo here, omit it in real applications
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240303_155004"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240303_155004/ds_sub_fit/sub_fit_ho.
2024-03-03 16:50:04,593	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning AutoGluon training ... Time limit 

In [ ]:
save_path = '/content/sample_data'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path, num_gpus=1).fit(train_data, presets='best_quality', num_stack_levels=3, num_bag_folds=10)

Get the best model used by AutoML tools.

In [8]:
predictor.get_model_best()

/tmp/ipykernel_12363/850388021.py:1: DeprecationWarning: `get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!
  predictor.get_model_best()


'WeightedEnsemble_L3'

Leaderboard based on the training data   
We can also use the customized metrics here by specifiying the extra_mertrics

In [10]:
predictor.leaderboard(extra_info = True, silent=True, extra_metrics = ['root_mean_squared_error'], data=valid_data)  

,model,score_test,root_mean_squared_error,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,NeuralNetTorch_BAG_L2,-174.167948,-174.167948,-156.358738,root_mean_squared_error,0.208874,0.199432,41.989415,0.104415,0.100705,...,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[OrgFertilizers, CropTillageDepth, RcNursEstDa...",None,"{'num_epochs': 20, 'epochs_wo_improve': 20, 'a...","{'batch_size': 64, 'num_epochs': 19}","{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[NeuralNetTorch_BAG_L1],[WeightedEnsemble_L3]
1,WeightedEnsemble_L3,-176.541643,-176.541643,-153.249601,root_mean_squared_error,0.209858,0.199682,42.032229,0.000984,0.000251,...,"{'use_orig_features': False, 'max_base_models'...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[NeuralNetTorch_BAG_L2, NeuralNetTorch_BAG_L1]",None,{'ensemble_size': 100},{'ensemble_size': 15},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[NeuralNetTorch_BAG_L2, NeuralNetTorch_BAG_L1]",[]
2,NeuralNetTorch_BAG_L1,-181.647965,-181.647965,-157.301221,root_mean_squared_error,0.104459,0.098726,21.040712,0.104459,0.098726,...,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[OrgFertilizers, CropTillageDepth, RcNursEstDa...",None,"{'num_epochs': 20, 'epochs_wo_improve': 20, 'a...","{'batch_size': 64, 'num_epochs': 19}","{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],"[NeuralNetTorch_BAG_L2, WeightedEnsemble_L2, W..."
3,WeightedEnsemble_L2,-181.647965,-181.647965,-157.301221,root_mean_squared_error,0.105213,0.098975,21.042385,0.000754,0.000249,...,"{'use_orig_features': False, 'max_base_models'...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[NeuralNetTorch_BAG_L1],None,{'ensemble_size': 100},{'ensemble_size': 1},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[NeuralNetTorch_BAG_L1],[]


Leaderboard based on the test data

In [11]:
predictor.leaderboard(valid_data, extra_info = True, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,NeuralNetTorch_BAG_L2,-174.167948,-156.358738,root_mean_squared_error,0.207926,0.199432,41.989415,0.104296,0.100705,20.948703,...,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[OrgFertilizers, CropTillageDepth, RcNursEstDa...",None,"{'num_epochs': 20, 'epochs_wo_improve': 20, 'a...","{'batch_size': 64, 'num_epochs': 19}","{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[NeuralNetTorch_BAG_L1],[WeightedEnsemble_L3]
1,WeightedEnsemble_L3,-176.541643,-153.249601,root_mean_squared_error,0.208890,0.199682,42.032229,0.000964,0.000251,0.042814,...,"{'use_orig_features': False, 'max_base_models'...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[NeuralNetTorch_BAG_L2, NeuralNetTorch_BAG_L1]",None,{'ensemble_size': 100},{'ensemble_size': 15},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[NeuralNetTorch_BAG_L2, NeuralNetTorch_BAG_L1]",[]
2,NeuralNetTorch_BAG_L1,-181.647965,-157.301221,root_mean_squared_error,0.103630,0.098726,21.040712,0.103630,0.098726,21.040712,...,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[OrgFertilizers, CropTillageDepth, RcNursEstDa...",None,"{'num_epochs': 20, 'epochs_wo_improve': 20, 'a...","{'batch_size': 64, 'num_epochs': 19}","{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],"[NeuralNetTorch_BAG_L2, WeightedEnsemble_L2, W..."
3,WeightedEnsemble_L2,-181.647965,-157.301221,root_mean_squared_error,0.104384,0.098975,21.042385,0.000754,0.000249,0.001673,...,"{'use_orig_features': False, 'max_base_models'...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[NeuralNetTorch_BAG_L1],None,{'ensemble_size': 100},{'ensemble_size': 1},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[NeuralNetTorch_BAG_L1],[]


In [12]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                   model   score_val              eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    WeightedEnsemble_L3 -153.249601  root_mean_squared_error       0.199682  42.032229                0.000251           0.042814            3       True          4
1  NeuralNetTorch_BAG_L2 -156.358738  root_mean_squared_error       0.199432  41.989415                0.100705          20.948703            2       True          3
2  NeuralNetTorch_BAG_L1 -157.301221  root_mean_squared_error       0.098726  21.040712                0.098726          21.040712            1       True          1
3    WeightedEnsemble_L2 -157.301221  root_mean_squared_error       0.098975  21.042385                0.000249           0.001673            2       True          2
Number of models trained: 4
Types of models trained:
{'WeightedEnsembleModel', 'StackerEnsembleModel_Tabular

/home/lmmista-wap266/anaconda3/envs/automl/lib/python3.9/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [13]:
all_models = predictor.get_model_names()
model_to_use = all_models[1]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

/tmp/ipykernel_12363/278995493.py:1: DeprecationWarning: `get_model_names` has been deprecated and will be removed in version 1.2. Please use `model_names` instead. This will raise an error in the future!
  all_models = predictor.get_model_names()


In [14]:
predictor.feature_importance(valid_data)

These features in provided data are not utilized by the predictor and will be ignored: ['ID', 'Residue_perc']
Computing feature importance via permutation shuffling for 41 features using 620 rows with 5 shuffle sets...
	52.59s	= Expected runtime (10.52s per shuffle set)
	11.57s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Acre,202.951906,7.507885,2.243299e-07,5,218.410745,187.493067
BasalDAP,21.892375,2.269117,1.365343e-05,5,26.564519,17.220232
Harv_hand_rent,19.058098,6.199579,1.173254e-03,5,31.823117,6.293080
1tdUrea,15.259450,4.913006,1.128892e-03,5,25.375397,5.143502
TransplantingIrrigationHours,8.219635,2.577771,1.022915e-03,5,13.527302,2.911968
LandPreparationMethod,7.578699,2.474369,1.189393e-03,5,12.673459,2.483938
Harv_date,7.497262,2.720987,1.761199e-03,5,13.099812,1.894712
Threshing_date,7.388973,2.118219,7.287487e-04,5,11.750415,3.027532
Block,7.194203,2.152182,8.563338e-04,5,11.625576,2.762831
CropCultLand,6.795520,1.947921,7.285066e-04,5,10.806317,2.784724
